In [0]:
import pandas as pd
import altair as alt

In [2]:
alt.data_transformers.enable('default', max_rows=None)

DataTransformerRegistry.enable('default')

In [3]:
crash1908 = pd.read_csv('https://gist.githubusercontent.com/myroslavarm/ce258616fa3e5da89c45d36ace0c45ae/raw/e4e93ca296a895dbd2145796d6751b90b40372a9/crashdata_since_1908.csv')
crash1908.head(20)

,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary
0,09/17/1908,17:18,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,1.0,0.0,"During a demonstration flight, a U.S. Army fly..."
1,07/12/1912,06:30,"AtlantiCity, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...
2,08/06/1913,NaN,"Victoria, British Columbia, Canada",Private,-,NaN,Curtiss seaplane,NaN,NaN,1.0,1.0,0.0,The first fatal airplane accident in Canada oc...
3,09/09/1913,18:30,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,20.0,14.0,0.0,The airship flew into a thunderstorm and encou...
4,10/17/1913,10:30,"Near Johannisthal, Germany",Military - German Navy,NaN,NaN,Zeppelin L-2 (airship),NaN,NaN,30.0,30.0,0.0,Hydrogen gas which was being vented was sucked...
5,03/05/1915,01:00,"Tienen, Belgium",Military - German Navy,NaN,NaN,Zeppelin L-8 (airship),NaN,NaN,41.0,21.0,0.0,Crashed into trees while attempting to land af...
6,09/03/1915,15:20,"Off Cuxhaven, Germany",Military - German Navy,NaN,NaN,Zeppelin L-10 (airship),NaN,NaN,19.0,19.0,0.0,"Exploded and burned near Neuwerk Island, when..."
7,07/28/1916,NaN,"Near Jambol, Bulgeria",Military - German Army,NaN,NaN,Schutte-Lanz S-L-10 (airship),NaN,NaN,20.0,20.0,0.0,"Crashed near the Black Sea, cause unknown."
8,09/24/1916,01:00,"Billericay, England",Military - German Navy,NaN,NaN,Zeppelin L-32 (airship),NaN,NaN,22.0,22.0,0.0,Shot down by British aircraft crashing in flames.
9,10/01/1916,23:45,"Potters Bar, England",Military - German Navy,NaN,NaN,Zeppelin L-31 (airship),NaN,NaN,19.0,19.0,0.0,Shot down in flames by the British 39th Home D...


In [4]:
crash1908.shape

(5268, 13)

In [5]:
crash1998 = pd.read_csv('https://gist.githubusercontent.com/myroslavarm/4afba6700af84453433133001dfe2d6d/raw/f53b2b7b0319a0cc460d99777a029e2bd445369d/crashdata_since_1998.csv')
crash1998.head()

,Information,Airlines,Manufacturar,Plane,Flight No.,passengers,killed
0,10 March 2019 An Ethiopian Airlines Boeing 737...,Ethiopian Airlines,Boeing,Boeing 737 Max 8,302,157,157
1,"29 October 2018 A Boeing 737 Max, operated by ...",Lion Air,Boeing,Boeing 737 Max 8,610,189,189
2,18 May 2018 A Boeing 737 passenger plane crash...,Global Air,Boeing,Boeing 737-201,972,113,112
3,11 April 2018 A military plane crashes shortly...,Algerian Air Force,Ilyushin,Ilyushin Il-76,7T-WIV,257,257
4,12 March 2018 A plane carrying 71 passengers a...,US-Bangla Airlines,Bombardier,Bombardier Dash 8 Q400,211,71,52


In [6]:
crash1998.shape

(95, 7)

In [7]:
aviation = pd.read_csv('https://gist.githubusercontent.com/myroslavarm/79863314ae2fa8c564bf9b98899d7be5/raw/b6096956ae693b74da66e6ada0ce49311074946a/aviationdata.csv')
aviation.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,Injury.Severity,Aircraft.Damage,Aircraft.Category,Registration.Number,Make,Model,Amateur.Built,Number.of.Engines,Engine.Type,FAR.Description,Schedule,Purpose.of.Flight,Air.Carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.Phase.of.Flight,Report.Status,Publication.Date
0,20181217X10943,Accident,GAA19CA098,2018-12-16,"Grangville, ID",United States,45.583611,-115.681667,PVT,NaN,Non-Fatal,Substantial,Airplane,N4385B,Cessna,170,No,1.0,NaN,Part 91: General Aviation,NaN,Positioning,NaN,NaN,NaN,NaN,1.0,NaN,NaN,Preliminary,2018-12-18
1,20181217X25746,Accident,GAA19CA097,2018-12-15,"MORIARTY, NM",United States,34.970000,-106.000000,NaN,MORIARTY MUNI,Non-Fatal,Substantial,Airplane,N188H,Cessna,180,No,1.0,NaN,Part 91: General Aviation,NaN,Personal,NaN,NaN,NaN,NaN,1.0,VMC,NaN,Preliminary,2018-12-20
2,20181213X41114,Accident,ERA19LA065,2018-12-13,"Punta Gorda, FL",United States,26.925278,-82.001111,PGD,Punta Gorda,Non-Fatal,Substantial,Airplane,N145MM,CZECH SPORT AIRCRAFT AS,Piper Sport,No,1.0,Reciprocating,Part 91: General Aviation,NaN,Personal,NaN,NaN,NaN,1.0,NaN,VMC,APPROACH,Preliminary,2018-12-21
3,20181213X45528,Accident,CEN19FA044,2018-12-13,"Valparaiso, IN",United States,41.451667,-87.004444,VPZ,Porter County Rgnl,Fatal(1),Destroyed,Airplane,N6785P,Piper,PA24,No,1.0,Reciprocating,Part 91: General Aviation,NaN,Personal,NaN,1.0,NaN,NaN,NaN,VMC,TAKEOFF,Preliminary,2018-12-19
4,20181214X90303,Accident,GAA19CA096,2018-12-11,"Hartford, WI",United States,43.330278,-88.326111,HXF,Hartford Muni,Non-Fatal,Substantial,Airplane,N6650M,Beech,36,No,1.0,NaN,Part 91: General Aviation,NaN,Personal,NaN,NaN,NaN,NaN,1.0,VMC,NaN,Preliminary,2018-12-18


In [8]:
aviation['Investigation.Type'].unique()
aviation['Injury.Severity'].unique()
aviation['Purpose.of.Flight'].unique()
aviation.shape

(82635, 31)

In [0]:
year = pd.DatetimeIndex(crash1908['Date']).year
crash1908 = crash1908.assign(Year=year.values)

In [10]:
crash = crash1908[crash1908['Year'].astype(int) >= 1960]
crash.head()

,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary,Year
1117,07/15/1966,NaN,Pakistan,Military - Pakistan Air Force,NaN,NaN,Lockheed C-130B Hercules,24142,3768,10.0,10.0,0.0,Struck a mountain.,1966
1237,02/13/1970,NaN,"Pin-la-Garenne, France",Avia-Taxi,NaN,NaN,Cessna 402,F-BPJA,402-0100,10.0,10.0,0.0,NaN,1970
1577,01/13/1962,NaN,"Near Tippi, Ethiopia",Ethiopian Airlines,NaN,Tippi - Jimma,Douglas DC-3,ET-T-1,10053,18.0,5.0,1.0,Swerved off the runway while taking off and st...,1962
1638,01/03/1960,10:40,"Taksing, India",Indian Airlines,NaN,NaN,Douglas C-47A,VT-CGG,12821,9.0,9.0,0.0,The cargo plane crashed while dropping supplie...,1960
1639,01/06/1960,02:38,"Wilmington/Bolivia, North Carolina",National Airlines,2511,New York - Miami,Douglas DC-6B,N8225H,43742/300,34.0,34.0,0.0,"The plane disintegrated in flight at 18,000 fe...",1960


In [11]:
crash = crash[~crash['Operator'].str.contains("Military|military", na=False)]
crash.shape

(3114, 14)

In [12]:
vis1 = crash.groupby(["Date", "Time", "Location", "Operator", "Aboard", "Fatalities"])['Ground'].count().reset_index()
vis1.head()

,Date,Time,Location,Operator,Aboard,Fatalities,Ground
0,01/01/1970,00:30,"Nulato, Alaska",Galena Air Service - Air Taxi,4.0,2.0,1
1,01/01/1974,12:38,"Near Turin, Italy",Itavia,42.0,38.0,1
2,01/01/1976,05:30,"Near Al Qaysumah, Saudi Arabia",Middle East Airlines,82.0,82.0,1
3,01/01/1978,20:15,"Off Bandra, Maharashtra, India",Air India,213.0,213.0,1
4,01/01/1985,20:30,"Near La Paz, Bolivia",Eastern Air Lines,29.0,29.0,1


In [0]:
vis1['Time'] = vis1.Time.str.replace("c: |c:",'')
vis1['Time'] = vis1.Time.str.replace("'",':')
vis1['Time'] = vis1.Time.str.replace(".",':')
vis1['Time'] = vis1.Time.str.replace("0943",'09:43')
vis1['Time'] = vis1.Time.str.replace("114:20",'14:20')

In [0]:
vis1['Time'] = pd.to_datetime(vis1.Time, format='%H:%M')

In [15]:
alt.Chart(vis1).mark_circle().encode(
    x = alt.X('year(Date):T', scale = alt.Scale(zero = False, domain=(1960, 2010))),
    y = alt.Y('hours(Time):O'),
    color=alt.value('red'),
    size = alt.Size('Fatalities:Q'),
    tooltip=['Location', 'Aboard', 'Fatalities']
).properties(width = 500)

alt.Chart(...)